# Coding Project Analysis JEM Forms


### Data Output: Date, Specimen ID, Rig Opertator, Layer, Cell Type Prediction (Human), Patch Duration(min)

In [1]:
import os
import csv
import json
import fnmatch
import numpy as np
import pandas as pd

import JEM_post_patch as pp #post patch script
#post_patch_column = 'extraction.postPatch'
#post_patch_pipette_column = 'extraction.endPipetteR'

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.io.json import json_normalize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
#from single_cell_ephys.lims_funcs import limsquery
#from single_cell_ephys.file_funcs import validated_input

In [2]:
pd.options.display.max_columns = None  #Displays all the columns 

In [3]:
#Used csv from 5/25/2018
shiny_human_df = pd.read_csv('C:/users/ramr/documents/github/personal-projects/csv/shiny_human.csv')
shiny_mouse_df = pd.read_csv('C:/users/ramr/documents/github/personal-projects/csv/shiny_mouse.csv')

In [4]:
#List of common functions(In progress)

def gen_fil(df, col, cond):
    '''
    Filters the column with a certain condition
    df: dataframe
    col: column
    cond: condition
    '''
    filtered = df[df[col] == cond]
    return filtered

def gen_cat(df, col):
    '''
    Assigns astype to the column
    df: dataframe
    col: column
    '''    
    category = df[col].astype('category')
    return category

def cell_count(P_num):
    '''
    Counts number of Lims tube id based on P# as number of cells
    df = dataframe
    count() = counts number
    '''
    user = df1[df1['Lims tube id'].str.contains(P_num)]
    return user['Lims tube id'].count()

def P_user(P_num):
    '''
    Prints cell_count 
    '''
    print("Total cells:", cell_count(P_num))

In [5]:
def get_jsons(dirname, expt):
    json_paths = []
    for jfile in os.listdir(dirname):
        if fnmatch.fnmatch(jfile, '*.%s.json' %expt):
            jpath = os.path.join(dirname, jfile)
            json_paths.append(jpath)
    return json_paths

In [6]:
#make list of json paths for every json file in MIES Experiments directory on 279
json_list = get_jsons("//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/", "PS")
#json_list

In [7]:
def flatten_attempts(slice_info, form_version):
    """Return flattened slice metadata dataframe.
    
    Parameters
    ----------
    slice_info : dict
        A dictionary of slices with nested pipette attempts.
    
    form_version : string
        A string containing the JEM form version.
        (Pre-version 2 contains IVSCC, PatchSeq and Electroporation arrays)
    Returns
    -------
    attempts_slice_df : pandas dataframe
        A dataframe of all pipette attempts along with all slice metadata.
    """
    
    df = json_normalize(slice_info)
    if form_version >= "2":
        ps_array_name = "pipettes"
    else:
        ps_array_name = "pipettesPatchSeqPilot"
    try:
        attempts_df = json_normalize(slice_info[ps_array_name])
        attempts_df["limsSpecName"] = df["limsSpecName"].values[0]
        attempts_df["attempt"] = [p+1 for p in attempts_df.index.values]
        attempts_slice_df = pd.merge(df, attempts_df, how="outer", on="limsSpecName")
        attempts_slice_df.drop(ps_array_name, axis=1, inplace=True)
        return attempts_slice_df
    except IndexError:
        pass

def is_field(df, colname):
    """Determine whether a column name exists in a dataframe.
    
    Parameters
    ----------
    df : a Pandas dataframe
    colname : string
        
    Returns
    -------
    Boolean
        Boolean value indicating if the colname exists in the dataframe.
    """
    
    try:
        df[colname]
        return True
    except KeyError:
        return False

### json_df

In [8]:
json_df = pd.DataFrame()

for json_path in json_list:
    with open(json_path) as data_file:
        slice_info = json.load(data_file)
        if is_field(slice_info, "formVersion"):
            jem_version = slice_info["formVersion"]
        else:
            jem_version = "1.0.0"
        flat_df = flatten_attempts(slice_info, jem_version)
        json_df = pd.concat([json_df, flat_df], axis=0)

In [ ]:
json_df.head()

In [9]:
#Creates patch_duration column and outputs in minutes
json_df['patch_duration'] = pd.to_datetime(json_df['extraction.timeRetractionEnd']) - pd.to_datetime(json_df['recording.timeWholeCellStart'])
json_df['patch_duration'] = (json_df['patch_duration'].astype('timedelta64[s]'))/60
json_df.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration
0,2018-04-18,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,1.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:20:14 -07:00,13:20:13 -07:00,13:20:14 -07:00,NaN,PXS4_180418_551_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:19:59 -07:00,13:20:00 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.233333
1,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,2.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_present,-80.0,-80.0,NaN,NaN,13:21:44 -07:00,13:21:44 -07:00,13:21:45 -07:00,NaN,PXS4_180418_552_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:21:26 -07:00,13:21:31 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.233333
2,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:22:39 -07:00,13:22:38 -07:00,13:22:39 -07:00,NaN,PXS4_180418_553_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:22:25 -07:00,13:22:27 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.200000
3,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,not_intentionally,nucleus_present,-80.0,-80.0,NaN,NaN,13:23:59 -07:00,13:23:58 -07:00,13:23:59 -07:00,NaN,PXS4_180418_554_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:23:04 -07:00,13:23:09 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.833333
4,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_absent,-80.0,-80.0,NaN,NaN,13:25:01 -07:00,13:25:01 -07:00,13:25:01 -07:00,NaN,PXS4_180418_555_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:24:37 -07:00,13:24:40 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN,0.350000


### Fixing up date column & adding post patch column

In [11]:
json_df['date'] = json_df['date'].str[:10] #Strip away the time
json_df['date'] = pd.to_datetime(json_df['date']) #Converting to YYYY-MM-DD
json_df.set_index('date', inplace = True)
json_df = pp.postpatch_reclass(json_df) #Makes post patch column with the script
json_df.sort_index(inplace = True) #Sort the date column just in case

#start_date = input('Enter the start date for your dataframe (YYYY-MM-DD): ')
start_date = '2017-10-01'
json_df1 = json_df[start_date:]
json_df1 = gen_fil(json_df1, 'status', 'SUCCESS') 
json_df1.sort_index(inplace = True) #Sort the date column just in case

In [12]:
json_df1.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration,post_patch
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,NaN,Fluorescence in Pipette,no,nucleus_absent,-32.5,-32.5,"Large Bubbles,Solution in Pipette Shank",NaN,14:48:35,14:47:57,14:51:55,14:48:36,053,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,14:41:05,14:42:42,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,9.216667,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,5.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,40,NaN,Fluorescence in Pipette,no,nucleus_absent,-35.0,-35.0,No Bubbles,NaN,16:07:20,16:06:38,16:13:41,16:07:21,056,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:59:05,16:01:26,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,12.250000,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,13.766667,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1400,2.5 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,14:28:51,14:28:20,14:31:39,14:29:08,355,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.02,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,14:19:45,14:21:02,7,lindsayn,NaN,NaN,SUCCESS,Patch/Cell Unstable,EPhys_171002_01_A05_M,10.616667,Nuc-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,3,3.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,NaN,no,nucleus_present,-23.9,-23.9,No Bubbles,NaN,14:41:51,14:41:34,14:43:42,14:41:56,402,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.03.01,NaN,NaN,NaN,NaN,NaN,NaN,5.2,NaN,14:35:37,14:36:25,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_A06_M,7.283333,Nuc-high seal


In [13]:
json_df1.tail()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration,post_patch
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-06-01,2018-05-31,NaN,NaN,4,NaN,Cre+,NaN,NaN,NaN,NaN,Electroporation,NaN,NaN,Channel recording,NaN,3,2.0,None,NaN,2018-05-11,37.8,NaN,NaN,1500,Alexa fill visible post experiment.,Fluorescence in Pipette,intentionally,nucleus_present,-35.0,-35.0,Small Bubbles,NaN,12:12:27 -07:00,12:11:41 -07:00,12:13:24 -07:00,NaN,P2S4_180601_052_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Sst-IRES-Cre;Ai14-392700.05.02,VISp5,NaN,NaN,NaN,NaN,NaN,4.4,NaN,11:54:21 -07:00,11:55:31 -07:00,1,rustym,#Schedule,Good,SUCCESS,NaN,NaN,17.883333,Nuc-high seal
2018-06-01,2018-05-31,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,3,3.0,None,NaN,2018-05-11,0,NaN,NaN,3000,NaN,Fluorescence in Pipette,no,nucleus_absent,-20.0,-20.0,No Bubbles,NaN,10:41:34 -07:00,10:41:18 -07:00,10:41:55 -07:00,NaN,P1S4_180601_002_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Sst-IRES-Cre;Ai14-392700.03.01,VISp5,NaN,NaN,NaN,NaN,NaN,4.6,NaN,10:37:35 -07:00,10:38:25 -07:00,4,kristenh,working outside WSE,Good,SUCCESS,NaN,NaN,3.500000,No-high seal
2018-06-01,2018-05-31,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-05-11,59,NaN,NaN,300,NaN,NaN,not_intentionally,nucleus_present,-49.0,-49.0,Medium Bubbles,NaN,12:36:30 -07:00,12:35:00 -07:00,12:36:31 -07:00,NaN,PBS4_180601_505_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Sst-IRES-Cre;Ai14-392700.05.01,VISp4,NaN,NaN,NaN,NaN,NaN,4.5,NaN,12:22:25 -07:00,12:23:48 -07:00,7,dijonh,#ROIGone,Good,SUCCESS,NaN,NaN,12.716667,Nuc-low seal
2018-06-01,2018-05-31,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-05-11,35,NaN,NaN,2000,NaN,NaN,no,nucleus_present,-38.0,-38.0,Small Bubbles,NaN,14:05:53 -07:00,14:04:26 -07:00,14:08:15 -07:00,NaN,P8S4_180601_356_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Th-P2A-FlpO;Ai65F-391841.04.02,VISp5,NaN,NaN,NaN,NaN,NaN,4.3,NaN,13:56:00 -07:00,13:57:09 -07:00,2,lindsayn,NaN,"Damaged,'Wave of Death'",SUCCESS,"Patch Became Leaky,Cell Depolarized",NaN,11.100000,Nuc-high seal
2018-06-01,2018-05-31,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,3,1.0,None,NaN,2018-05-11,43,NaN,NaN,1000,NaN,Cell Swelled,no,nucleus_present,-35.0,-35.0,No Bubbles,NaN,13:29:53 -07:00,13:29:02 -07:00,13:31:03 -07:00,NaN,P8S4_180601_355_A01,NaN,NaN,Yes,2.0.3,NaN,2018-04-09,NaN,NaN,NaN,NaN,NaN,Th-P2A-FlpO;Ai65F-391841.04.02,VISp4,NaN,NaN,NaN,NaN,NaN,4.4,NaN,13:21:52 -07:00,13:23:09 -07:00,2,lindsayn,NaN,"Damaged,'Wave of Death'",SUCCESS,Patch Became Leaky,NaN,7.900000,Nuc-high seal


### Filter for Human & Mouse dataframe

In [14]:
json_df1['human?'] = json_df1['limsSpecName'].str.match(r"H\d\d") #Creates new boolean column

json_df2 = gen_fil(json_df1, 'human?', True) #HUMAN
json_df3 = gen_fil(json_df1, 'human?', False) #MOUSE
json_df1.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,patch_duration,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,NaN,Fluorescence in Pipette,no,nucleus_absent,-32.5,-32.5,"Large Bubbles,Solution in Pipette Shank",NaN,14:48:35,14:47:57,14:51:55,14:48:36,053,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,14:41:05,14:42:42,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,9.216667,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,5.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,40,NaN,Fluorescence in Pipette,no,nucleus_absent,-35.0,-35.0,No Bubbles,NaN,16:07:20,16:06:38,16:13:41,16:07:21,056,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:59:05,16:01:26,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,12.250000,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,13.766667,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1400,2.5 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,14:28:51,14:28:20,14:31:39,14:29:08,355,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.02,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,14:19:45,14:21:02,7,lindsayn,NaN,NaN,SUCCESS,Patch/Cell Unstable,EPhys_171002_01_A05_M,10.616667,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,3,3.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,NaN,no,nucleus_present,-23.9,-23.9,No Bubbles,NaN,14:41:51,14:41:34,14:43:42,14:41:56,402,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.03.01,NaN,NaN,NaN,NaN,NaN,NaN,5.2,NaN,14:35:37,14:36:25,6,lisak,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_A06_M,7.283333,Nuc-high seal,False


## HUMAN dataframe

In [ ]:
json_df2 = json_df2.reset_index()
human_df = json_df2[['date',
                     'limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'recording.humanCellTypePrediction',
                     'post_patch',
                     'patch_duration',
                     'extraction.tubeID']]

human_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-10 to 2017-11-27
#manualRoi avaible data at 2017-11-27 to Present
#extraction.tubeID changes from ### to User_Date_###_A01 at 2017-11-27

shiny_human_df = shiny_human_df.loc[:,['sample_id',
                                    'cell_name_label',
                                    'res_index_label']]

final_H_df = pd.merge(left = human_df, 
                      right = shiny_human_df, 
                      left_on = 'extraction.tubeID', 
                      right_on = 'sample_id', 
                      how = 'inner')

In [ ]:
final_H_df.head()

In [ ]:
final_H_df.tail()

In [ ]:
human_df['post_patch'].value_counts()

In [ ]:
final_H_df['post_patch'].value_counts()

## MOUSE dataframe

In [ ]:
json_df3 = json_df3.reset_index()
mouse_df = json_df3[['date',
                     'limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'post_patch',
                     'patch_duration',
                     'extraction.tubeID']]

mouse_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-02 to 2017-11-10
#manualRoi avaible data at 2017-11-10 to Present
#extraction.tubeID changes from ### to User_Date_###_A01 at 2017-11-10

shiny_mouse_df = shiny_mouse_df.loc[:,['sample_id',
                                       'cell_name_label',
                                       'res_index_label']]

final_M_df = pd.merge(left = mouse_df, 
                      right = shiny_mouse_df, 
                      left_on = 'extraction.tubeID', 
                      right_on = 'sample_id', 
                      how = 'inner')

In [ ]:
final_M_df.head()

In [ ]:
final_M_df.tail()

In [ ]:
mouse_df['post_patch'].value_counts()

In [ ]:
final_M_df['post_patch'].value_counts()

## Extra

In [ ]:
def nuc_count(df):
    #user = str(input('Enter the user number (P#): '))
    user = 'PA'
    
    nh = df[df['post_patch'].str.contains('Nuc-high seal') & 
            df['extraction.tubeID'].str.contains(user)]
    nh = nh['post_patch'].count()

    nl = df[df['post_patch'].str.contains('Nuc-low seal') & 
            df['extraction.tubeID'].str.contains(user)]
    nl = nl['post_patch'].count()

    noh = df[df['post_patch'].str.contains('No-high seal') & 
             df['extraction.tubeID'].str.contains(user)]
    noh = noh['post_patch'].count()

    nol = df[df['post_patch'].str.contains('No-low seal') & 
             df['extraction.tubeID'].str.contains(user)]
    nol = nol['post_patch'].count()

    en = df[df['post_patch'].str.contains('Entire cell') & 
            df['extraction.tubeID'].str.contains(user)]
    en = en['post_patch'].count()
    

    print 'Nucleated:', nh
    print 'Partial nucleated', nl
    print 'Outside-out:', noh
    print 'No seal:', nol
    print 'Entire cell:', en   

In [ ]:
nuc_count(mouse_df)

In [ ]:
nuc_count(final_M_df)

In [ ]:
nuc_count(human_df)

In [ ]:
nuc_count(final_H_df)

In [ ]:
#Problem: index date column doesn't show dates after forming new combined df with shiny